In [16]:
# 1. Select a pool of large, liquid, optionable, dividend-yielding stocks
# 2. Collect the expected dividend and ex-dividend dates of these stocks
# 3. Select an options month that is after the ex-dividend date
# 4. Select an OTM option that provides a balance of liquidity, upside, and credit

In [49]:
import pandas as pd

roundup_df = pd.read_csv("2018-09-25_mid-safe-u50.csv")
roundup_df.columns

Index(['Ticker', 'No.', 'Market Cap', 'Dividend %', 'ROA %', 'ROE %', 'ROI %',
       'Curr R', 'Quick R', 'LTDebt/Eq', 'Debt/Eq', 'Gross M %', 'Oper M %',
       'Profit M %', 'Earnings', 'Price', 'Change %', 'Volume', 'Put Strike',
       'Put Expiration', 'Put Last', 'Put Chg', 'Put Bid', 'Put Ask',
       'Put Vol', 'Put Open', 'Call Expiration', 'Call Last', 'Call Chg',
       'Call Bid', 'Call Ask', 'Call Vol', 'Call Open', 'Call Strike',
       'Ex-Div Date (est)', 'Div Amount (est)', 'Dividend Period'],
      dtype='object')

In [50]:
import datetime
import numpy as np
focus_cols = ["Ticker", "Market Cap", "Earnings (est)",
              "Ex-Div Date (est)", "Price", "Div Amount (est)",
              "Call Expiration", "Call Strike", "Call Last",
             "Put Expiration", "Put Strike", "Put Last"]

roundup_df["Earnings"] = pd.to_datetime(roundup_df["Earnings"])
earnings_proj = roundup_df["Earnings"] + roundup_df["Dividend Period"].apply(lambda d: datetime.timedelta(days=d))
today = datetime.date.today()
roundup_df["Earnings (est)"] = np.where(roundup_df["Earnings"] < today, earnings_proj, roundup_df["Earnings"])

focused = roundup_df[focus_cols].copy()
focused = focused[~focused["Put Strike"].isnull() & ~focused["Call Strike"].isnull()]
focused

,Ticker,Market Cap,Earnings (est),Ex-Div Date (est),Price,Div Amount (est),Call Expiration,Call Strike,Call Last,Put Expiration,Put Strike,Put Last
0,VLP,2.580000e+09,2018-10-25,2018-11-04,36.74,0.551,2019-02-15,40.0,0.60,2018-10-19,35,0.40
1,CQP,1.949000e+10,2018-11-13,2018-11-02,39.84,0.560,2018-12-21,42.0,0.47,2018-10-19,39,0.95
6,TPR,1.423000e+10,2018-11-13,2018-12-06,49.25,0.338,2019-01-18,52.5,1.55,2018-11-16,45,0.70
7,ADM,2.779000e+10,2018-10-30,2018-11-14,49.64,0.335,2018-12-21,52.5,0.63,2018-11-02,49,0.69
8,TRCO,3.330000e+09,2018-11-07,2018-11-16,37.99,0.250,2019-01-18,40.0,1.91,2018-10-19,35,0.40
9,INTC,2.141400e+11,2018-10-25,2018-11-06,45.91,0.300,2018-12-21,50.0,0.76,2018-10-19,42,0.13
10,RES,3.400000e+09,2018-10-24,2018-11-09,16.02,0.100,2018-12-21,17.0,0.80,2018-10-19,15,0.25
12,DKS,3.640000e+09,2018-12-05,2018-12-20,36.76,0.225,2019-01-18,40.0,1.80,2018-11-16,36,1.35
13,JNPR,1.020000e+10,2018-10-23,2018-12-01,29.79,0.180,2019-01-18,32.0,0.78,2018-11-02,28,0.57
14,CA,1.840000e+10,2018-11-05,2018-11-28,44.06,0.255,2019-02-15,47.0,NaN,2018-11-16,43,0.05


In [51]:
perf = focused[["Ticker", "Earnings (est)", "Ex-Div Date (est)"]].copy()
perf["P Margin"] = focused["Put Strike"] * 100
perf["P Credit %"] = focused["Put Last"] / focused["Put Strike"] * 100
perf["P Downside %"] = (focused["Price"] - focused["Put Strike"]) / focused["Price"] * 100
perf["CC Margin"] = (focused["Price"] - focused["Call Last"]) * 100
perf["D Credit %"] = focused["Div Amount (est)"] * 100 / perf["CC Margin"] * 100
perf["C Credit %"] = focused["Call Last"] * 100 / perf["CC Margin"] * 100
perf["C Upside %"] = (focused["Call Strike"] - focused["Price"]) / focused["Price"] * 100
# focused["DTE"] = (focused["Call Expiration"] - today).astype('timedelta64[D]')
# focused["Early Exercise Days"] = (focused["Call Expiration"] - focused["Ex-Div Date (est)"]).astype('timedelta64[D]')

perf

,Ticker,Earnings (est),Ex-Div Date (est),P Margin,P Credit %,P Downside %,CC Margin,D Credit %,C Credit %,C Upside %
0,VLP,2018-10-25,2018-11-04,3500,1.142857,4.735983,3614,1.524626,1.660210,8.873163
1,CQP,2018-11-13,2018-11-02,3900,2.435897,2.108434,3937,1.422403,1.193802,5.421687
6,TPR,2018-11-13,2018-12-06,4500,1.555556,8.629442,4770,0.708595,3.249476,6.598985
7,ADM,2018-10-30,2018-11-14,4900,1.408163,1.289283,4901,0.683534,1.285452,5.761483
8,TRCO,2018-11-07,2018-11-16,3500,1.142857,7.870492,3608,0.692905,5.293792,5.290866
9,INTC,2018-10-25,2018-11-06,4200,0.309524,8.516663,4515,0.664452,1.683278,8.908734
10,RES,2018-10-24,2018-11-09,1500,1.666667,6.367041,1522,0.657030,5.256242,6.117353
12,DKS,2018-12-05,2018-12-20,3600,3.750000,2.067465,3496,0.643593,5.148741,8.813928
13,JNPR,2018-10-23,2018-12-01,2800,2.035714,6.008728,2901,0.620476,2.688728,7.418597
14,CA,2018-11-05,2018-11-28,4300,0.116279,2.405810,NaN,NaN,NaN,6.672719
